# Generic Mapping Tools - GMT

GMT is one of the commonly used scripting based map plotting packages in geophysics and also other disciplines. GMT itself has to be scripted in `bash`. Nevertheless also GMT wrappers for Python, Julia and Matlab were released through the last years.

## Installation

One of largest challenges of GMT is its installation. In our case we are going to use GMT version 6.4, for which its python wrapping `pyGMT` is available.

<div class='alert alert-info'>
    <p style="font-weight: bold; font-size: 150%">Good to know:</p>
    PyGMT is just one wrapper. There are others, which aid you in using GMT within Python as e.g. Automap and GMTPY (modules within Pyrocko). If you are interested, check out <a href='https://pyrocko.org/docs/current/library/examples/plotting.html#generating-topographic-maps-with-automap'>pyrocko.org/docs/current/library/examples/plotting.html</a>
</div>


Depending on the environment of Python you are using, you can install it via `conda` or have to install from source.
Installing instructions for **GMT** are located at [github.com\/GenericMappingTools/gmt/blob/master/BUILDING.md](https://github.com/GenericMappingTools/gmt/blob/master/BUILDING.md). The source packages and additional data can be found at [github.com\/GenericMappingTools/gmt/releases](https://github.com/GenericMappingTools/gmt/releases) and [generic-mapping-tools.org\/download/](https://www.generic-mapping-tools.org/download/)

**PyGMT** has its own installation instructions at [pygmt.org\/latest/install.html](https://www.pygmt.org/latest/install.html).


## Test
Let's hope, we have accomplished the parcour through the installations and can test the basic functionality:

In [ ]:
# Import of pygmt package (and numpy for later use)
import pygmt
import numpy as num

# List detailed information on package
pygmt.show_versions()

# Plot first map
fig = pygmt.Figure()
fig.coast(
    region="g",
    frame=True,
    shorelines=1)
fig.show()

## Basic Map Generation (World, Germany) - Projections

We have successfully generated our first figure, but hey, it does not really look super appealing, right? We should start tweaking our figure by adopting the projection, so the way, the earth (a 3D sphere) is converted into a 2D map. The available projections are listed within the PyGMT docs at [pygmt.org\/latest/projections/index.html](https://www.pygmt.org/latest/projections/index.html).

The first task will be to choose the `Mollweide` projection and replot the map above in this projection.

In [ ]:
fig = pygmt.Figure()

fig.coast(
    region="g",
    projection="W10c", # set the projection to Mollweide
    frame=True,
    shorelines=1)
fig.show()

Hint: You can find the full documentation of the `coast` command here: [pygmt.org\/latest/api/generated/pygmt.Figure.basemap.html#pygmt.Figure.basemap](https://www.pygmt.org/latest/api/generated/pygmt.Figure.basemap.html#pygmt.Figure.basemap).

Great, we are able to plot some basic global maps. But how about a more focused map:

In [ ]:
fig = pygmt.Figure()

# Define region of the map
region = [6, 20, 36, 48]  # geographical limits [lon_min, lon_max, lat_min, lat_max]

# Generate basic map
fig.coast(
    region=region,
    projection="M15c",  # Definition of projection
    frame=True,  # If True, our map shows a frame
    shorelines=1, # Which coast lines are shown, here major lakes, rivers and seashores
    resolution='c')  # Resolution of the coast line: test ['c', 'l', 'i', 'h', 'f']
fig.show()

Hm, better but why does it look so weird. So here comes your first task:

<div class='alert alert-success'>
    <p style="font-weight: bold; font-size: 150%">Task 1:</p>
    <ol>
        <li>Check the parameter <em>resolution</em> and test the different available options. What does change?</li>
    </ol>
    <span style='font-weight: bold'>Hint:</span>
Check <a href='https://www.pygmt.org/latest/tutorials/basics/coastlines.html#sphx-glr-tutorials-basics-coastlines-py'>the PyGMT coastline tutorial.</a>
</div>


Have you accomplished the task? Great! But even now there are still options for improvement. Therefore we have to turn things a bit more advanced and create now a figure with colored land and water surfaces:

In [ ]:
fig = pygmt.Figure()

# Generate general empty plot frame (similar to examples before)
fig.basemap(region=region, projection="M15c", frame=True)

# Plot the coast and rivers with specific colors
fig.coast(
    land="#666666",  # Color for land surfaces
    water="skyblue",  # Color for water surfaces
    shorelines=1)

fig.show()

## Features

So, we have generated a simple, rather ok looking map of Italy. We can display land and sea surfaces and coast lines. GMT provides us with some additional data, which we can now use to add some information to our map. 

### National borders

As said, the map shows already some essentials. For easier orientation on the map, national borders are often helpful. GMTs' `coast` command allows to plot border rather simple, adding only one new line of code:


In [ ]:
fig = pygmt.Figure()

# Generate general empty plot frame
fig.basemap(
    region=[6, 20, 36, 48],
    projection="M15c",
    frame=True)

# Plot the coast and rivers with specific colors
fig.coast(
    land="#666666",
    water="skyblue",
    shorelines=1,
    borders="1/2p,darkred,-") # Add national borders (1) with 2 points linewidth, darkred color and dashed line

fig.show()

<div class='alert alert-success'>
    <p style="font-weight: bold; font-size: 150%">Task 2:</p>
    <ol>
        <li>Adopt the appearance of the borders and change the line size, color and line style</li>
    </ol>
    <span style='font-weight: bold'>Hint:</span>
Check <a href='https://www.pygmt.org/latest/gallery/lines/linestyles.html'>the PyGMT line style guide.</a>
</div>


### Rivers
Similar to national borders we can also add rivers. 

In [ ]:
fig = pygmt.Figure()

# Generate general empty plot frame
fig.basemap(region=region, projection="M15c", frame=True)

# Plot the coast and rivers with specific colors
fig.coast(
    land="#666666",
    water="skyblue",
    shorelines=1,
    borders="1/2p,darkred,-",
    rivers='r/0.5p,darkblue')  # Draw major rivers with 0.5 point linewidth in darkblue

fig.show()

### Topography

Topographical datasets are built-in in PyGMT, but need to be downloaded at the first use. PyGMT offers datasets in different resolutions ([check the documentation](https://www.pygmt.org/latest/api/generated/pygmt.datasets.load_earth_relief.html#pygmt.datasets.load_earth_relief)). The better the resolution, the longer the download. Here we will use 15 second data.

The flow of topographic plotting consists of three steps:
1. loading the topography in the desired resolution for the desired region
1. plotting the topography with a specified colormap
1. plotting the associated colorbar

In [ ]:
fig = pygmt.Figure()

# Generate general empty plot frame
fig.basemap(region=region, projection="M15c", frame=True)

# Load topography
grid = pygmt.datasets.load_earth_relief(
    resolution="15s",  # Resolution of 15 arc seconds
    region=region)

# Plot topography
fig.grdimage(
    grid=grid,
    cmap="geo")  # Use the "geo" colormap

# Plot colorbar with some specifications
fig.colorbar(
    frame=[
        "a1000",  # Increment every 1000 m
        "x+lElevation", # Set "Elevation" as Title
        "y+lm"])  # Set "m" as colorbar unit

# Plot the coast and rivers with specific colors
fig.coast(shorelines=1, borders="1/2p,darkred,-", rivers='r/1p,darkblue')

fig.show()

## Symbols and labels

Locations, e.g. of cities, seismic stations, earthquakes, can be indicated by symbols and associated labels. 

In the following we will plot the location of Rome and also label it on the map:
Rome is located at 41.893333 N, 12.482778 E.

Check also the tutorials to plot symbols and text here:
* https://www.pygmt.org/latest/gallery/index.html#symbols-and-markers
* https://www.pygmt.org/latest/tutorials/basics/text.html
* https://www.pygmt.org/latest/tutorials/basics/plot.html

In [ ]:
fig = pygmt.Figure()

# Generate general empty plot frame
fig.basemap(region=region, projection="M15c", frame=True)

# Plot topography
fig.grdimage(grid=grid, cmap="geo")

# Plot colorbar
fig.colorbar(frame=["a1000", "x+lElevation", "y+lm"])

# Plot the coast and rivers with specific colors
fig.coast(shorelines=1, borders="1/2p,darkred,-", rivers='r/1p,darkblue')

# Coordinates of Rome
lat = num.array([41.893333])
lon = num.array([12.482778])

# Plot dot at location of Rome
fig.plot(
    x=lon,
    y=lat,
    style="c0.9c",  # Plot circle with 0.9 centimeter diameter
    pen="1p,black",  # Draw the boundary of the circle with 1 point line in black
    color="darkred")  # Circle fill color = darkred

# Plot label
fig.text(
    x=lon-0.5,
    y=lat+0.4,
    text='Rome',  # Label text
    font="15p,Helvetica-Bold,beige",  # Font specifications (size, Font, color)
    justify='MR',  # Justification relative to location ('MR' - middle right)
    angle=315)  # Rotate label by 315 degrees

fig.show()

Great, if you have accomplished these, there is now a special task:

<div class='alert alert-success'>
    <p style="font-weight: bold; font-size: 150%">Task 3:</p>
    <ol>
        <li>Plot the location and name of the 4 active volcanoes in Italy on the map using <em>plot</em> and <em>text</em>.</li>
        <li>If you want to escalate, use the volcano symbol.</li>
        <li>If you want to escalate even more, let the symbol scale with the height of the volcano.</li>
    </ol>
    <span style='font-weight: bold'>Hint:</span>
Check <a href='https://www.pygmt.org/latest/gallery/symbols/basic_symbols.html#sphx-glr-gallery-symbols-basic-symbols-py'>pygmt.org/latest/gallery/symbols/basic_symbols.html</a> and <a href='https://www.pygmt.org/latest/gallery/symbols/custom_symbols.html#sphx-glr-gallery-symbols-custom-symbols-py'>pygmt.org/latest/gallery/symbols/custom_symbols.html</a>.
</div>


In [ ]:
fig = pygmt.Figure()

# Generate general empty plot frame
fig.basemap(region=region, projection="M15c", frame=True)

# Plot topography
fig.grdimage(grid=grid, cmap="geo")

# Plot colorbar
fig.colorbar(frame=["a1000", "x+lElevation", "y+lm"])

# Plot the coast and rivers with specific colors
fig.coast(shorelines=1, borders="1/2p,darkred,-", rivers='r/1p,darkblue')

# Volcano coordinates, names and heights
volcanoe_lat = num.array([40.816667, 37.755, 38.793889, 38.399434])
volcanoe_lon = num.array([14.433333, 14.995, 15.211111, 14.963955])
volcanoe_height = num.array([1281.0, 3357.0, 924.0, 501.0])
volcanoe_name = num.array(['Vesuvius', 'Etna', 'Stromboli', 'Vulcano'])

# Plot the volcano locations

# Plot the volcano labels

fig.show()

## Contoured data

The final exercise is on contoured data. In the plots above topography has been plotted using `grdimage`. Now bathymetry (so everything below 0 m height) shall be plotted with contour lines every 1000 m (annotation every 2000 m).

We have do to several steps:
1. generate a grid which is NaN below zero
1. plot the topography with transparent NaN
1. plot the bathymetric contour lines

In [ ]:
fig = pygmt.Figure()

# Generate general empty plot frame
fig.basemap(region=region, projection="M15c", frame=True)

# Cut a specific grid for topography above sealevel and replace each value below 0 with NaN
grid_above_sl = pygmt.grdclip(
    grid,
    below=[0, num.nan])  # Replace everything below 0 with NaN

# Plot the new clipped topography
fig.grdimage(
    grid=grid_above_sl,
    cmap="geo",
    nan_transparent=True)  # Make NaN values (below sealevel) transparent

# Plot the coast and rivers with specific colors
fig.coast(shorelines=1, borders="1/2p,darkred,-", rivers='r/1p,darkblue')

# Plot bathymetry as contour
fig.grdcontour(
    grid=grid,
    annotation=2000,  # Annotation every 2000 m
    interval=1000,  # Draw lines every 1000 m
    limit=[-6000., 0.])  # Limit the drawn contour lines between -6000. and 0 Meter

fig.show()